In [17]:
import numpy as np

In [18]:
# DAG provided in appendix A
dag = {
    (0, 30), (1, 0), (2, 7), (3, 2), (4, 1), (5, 15), (6, 5), (7, 6), (8, 7), (9, 8), (10, 0), (11, 4), (12, 11), (13, 12), (16, 14), (14, 10), (15, 4), (16, 15), (17, 16), (18, 17), (19, 18), (20, 17), (21, 20), (22, 21), (23, 4), (24, 23), (25, 24), (26, 25), (27, 25), (28, 26), (28, 27), (29, 3), (29, 9),
    (29, 13), (29, 19), (29, 22), (29, 28)
}

# processing times provided in Appendix A
p = [3, 10, 2, 2, 5, 2, 14, 5, 6, 5, 5, 2, 3, 3, 5, 6, 6, 6, 2, 3, 2, 3, 14, 5, 18, 10, 2, 3, 6, 2, 10]

# due dates provided in Appendix A
d = [172, 82, 18, 61, 93, 71, 217, 295, 290, 287, 253, 307, 279, 73, 355, 34, 233, 77, 88, 122, 71, 181, 340, 141, 209, 217, 256, 144, 307, 329, 269]

num_nodes = len(p)

In [23]:
star_line = "*****************************************************************************************************************************************\n"

# Constructs an inverted graph by adjacency list and num_successors from a set of edges.
def build_graph(edges):

    inv_graph = dict()
    num_successors = dict()
    
    for u, v in edges:
        if u not in num_successors:
            num_successors[u] = 0
        if v not in num_successors:
            num_successors[v] = 0

        # Add nodes and edges to the graph
        if v not in inv_graph: inv_graph[v] = []
        inv_graph[v].append(u)
        num_successors[u] += 1
        
    return inv_graph, num_successors

# Calculates the tardiness giving a completion time and due date
def tardiness(C, dd): return max(0, C-dd)
    

In [33]:
# Finds an optimal schedule according to LCL given a graph, processing times, and due dates
def least_cost_last(dag, processing_times, due_dates):

    # build adjacency list and track number of successors
    inv_graph, num_successors = build_graph(dag)

    schedule = []
    total_processing_time = sum(processing_times)
    total_cost = 0

    # Jobs to be scheduled are ones with 0 successors
    ready_to_schedule = {job for job in num_successors if num_successors[job] == 0}
    count = 0
    printout = ""
    # While we have jobs to be scheduled left
    while ready_to_schedule:
        count += 1
        # Choose the job with the min tardiness
        last_job = min(ready_to_schedule, key=lambda j: (tardiness(total_processing_time, due_dates[j]), j))

        schedule.append(last_job)
        total_cost = max(total_cost, tardiness(total_processing_time, due_dates[last_job]))
        ready_to_schedule.remove(last_job)
        total_processing_time -= processing_times[last_job]
        
        # If the job is a root in the graph
        if last_job not in inv_graph: continue

        for parent in inv_graph[last_job]:

            # Decrement number of successors because we scheduled the job
            num_successors[parent] -= 1

            # If ready to be scheduled, add to list
            if num_successors[parent] == 0:  
                ready_to_schedule.add(parent)

        if count in {1, 2, 5, 10, 15, 20, 25}:
            print('Iteration number {0}: Schedule is '.format(count), [1 + n for n in schedule[::-1]])
            print('Cost is', total_cost)
            printout += 'Iteration number {0}: Schedule is '.format(count) + str([1 + n for n in schedule[::-1]]) + '\n'
            printout += 'Cost is ' + str(total_cost) +'\n'
            printout += star_line
        
    print('Final Schedule is ', [1 + n for n in schedule[::-1]])
    printout += 'Final Schedule is ' + str([1 + n for n in schedule[::-1]]) + '\n'
    printout += 'Cost is ' + str(total_cost) +'\n'
    printout += star_line
    
    return [1 + n for n in schedule[::-1]], total_cost, printout

In [34]:
schedule, cost, printout = least_cost_last(dag, p, d)
print('Cost is', cost)

Iteration number 1: Schedule is  [31]
Cost is 0
Iteration number 2: Schedule is  [1, 31]
Cost is 0
Iteration number 5: Schedule is  [2, 15, 11, 1, 31]
Cost is 65
Iteration number 10: Schedule is  [25, 24, 13, 12, 5, 2, 15, 11, 1, 31]
Cost is 65
Iteration number 15: Schedule is  [14, 29, 28, 27, 26, 25, 24, 13, 12, 5, 2, 15, 11, 1, 31]
Cost is 65
Iteration number 20: Schedule is  [8, 7, 6, 17, 16, 14, 29, 28, 27, 26, 25, 24, 13, 12, 5, 2, 15, 11, 1, 31]
Cost is 65
Iteration number 25: Schedule is  [20, 19, 18, 10, 9, 8, 7, 6, 17, 16, 14, 29, 28, 27, 26, 25, 24, 13, 12, 5, 2, 15, 11, 1, 31]
Cost is 65
Final Schedule is  [30, 4, 3, 23, 22, 21, 20, 19, 18, 10, 9, 8, 7, 6, 17, 16, 14, 29, 28, 27, 26, 25, 24, 13, 12, 5, 2, 15, 11, 1, 31]
Cost is 65


In [35]:
with open("printout_lcl.txt", "w") as file:
    # Write some text to the file
    file.write("This is a printout of the iterations for the Least Cost Last Rule\n")
    file.write(star_line)
    file.write(printout)

print("Data has been written to printout_lcl.txt")

Data has been written to printout_lcl.txt
